In [1]:
import time
from datetime import timedelta
import sys
import os # used for navigating to image path
import numpy as np
import shutil
import pandas as pd
import csv
from PIL import Image
import imageio # used for writing images
from sklearn.model_selection import train_test_split
sys.path.append(os.path.abspath('../python'))
from helpers import *

In [2]:
SEED = 100
IMAGE_WIDTH_LIST = [189, 252, 336]
CONF_DIR = '../../data/tidy/conflict-detection/'
ORIGINAL_TILES = os.path.join(CONF_DIR,'split-images/all_tiles/')
CLASSIFIED_TILES_DIR = os.path.join(CONF_DIR, 'classified-tiles/')
PREPROCESSED_TILES_DIR = os.path.join(CONF_DIR, 'preprocessed-tiles/')
TILES_ASSIGNMENT = os.path.join(CONF_DIR, 'split-images/tiles_assignment.csv')

In [3]:
df1 = pd.read_csv(TILES_ASSIGNMENT)

In [4]:
df1.head()

,Filename,Classification
0,improbable-455_0_0,NaN
1,improbable-455_0_336,NaN
2,improbable-455_0_672,NaN
3,improbable-455_0_1008,NaN
4,improbable-455_0_1344,NaN


In [5]:
def assign_sample_tiles(unassigned_tiles_path):
    """Randomly classifies tiles as 'c'(conflict) or 'n'(no conflict)"""
    np.random.seed(SEED) # ensure repeatability
    data = pd.read_csv(unassigned_tiles_path)
    data['Classification'] = np.random.choice(['c','n'], size=len(data))
    
    return(data)

In [6]:
# Assigned random tiles
df2 = assign_sample_tiles(TILES_ASSIGNMENT)

In [7]:
def generate_index(sample_data): #path_to_tiles_assignment
    """Serially labels all tiles by class and generates an index to data/tidy/conflict_split_images"""
    
#     data = pd.read_csv(path_to_tiles_assignment)
#     data = data.drop([0])
    
    df = pd.DataFrame(columns=['original', 'tile', 'classification'])
    
    counts = {'conflict':1,
             'no_conflict':1,
             'unknown':1}
        
    for row in sample_data.itertuples():
        tile = row.Filename
        original_image = str(os.path.splitext(tile)).split('_')[0][2:]
        classification = row.Classification

        if classification.lower() == 'c':        
            save_label = 'conflict' + '-' + str(counts['conflict']) + '.jpg'
            df.loc[row.Index] = [original_image] + [tile] + [save_label]
            counts['conflict'] += 1
        elif classification.lower() == 'n':
            save_label = 'no_conflict' + '-' + str(counts['no_conflict']) + '.jpg'
            df.loc[row.Index] = [original_image] + [tile] + [save_label]
            counts['no_conflict'] += 1
        else:
            save_label = 'unknown' + '-' + str(counts['unknown']) + '.jpg'
            df.loc[row.Index] = [original_image] + [tile] + [save_label]
            counts['unknown'] += 1
    
    df.to_csv(CONF_DIR+'tile_index_mapping.csv', encoding='utf-8', index=False)
    
    counts['conflict'] -= 1
    counts['no_conflict'] -= 1
    counts['unknown'] -= 1 
    
    print('Number of conflict tiles recorded:', counts['conflict'])    
    print('Number of no-conflict tiles recorded:', counts['no_conflict'])
    print('Number of unknown tiles:', counts['unknown'])
        
    return df, counts

In [8]:
df, c = generate_index(df2)

Number of conflict tiles recorded: 3085
Number of no-conflict tiles recorded: 2990
Number of unknown tiles: 0


In [9]:
df

,original,tile,classification
0,improbable-455,improbable-455_0_0,conflict-1.jpg
1,improbable-455,improbable-455_0_336,conflict-2.jpg
2,improbable-455,improbable-455_0_672,no_conflict-1.jpg
3,improbable-455,improbable-455_0_1008,no_conflict-2.jpg
4,improbable-455,improbable-455_0_1344,no_conflict-3.jpg
...,...,...,...
6070,probable-96,probable-96_3584_1344,conflict-3082.jpg
6071,probable-96,probable-96_3584_1680,conflict-3083.jpg
6072,probable-96,probable-96_3584_2016,conflict-3084.jpg
6073,probable-96,probable-96_3584_2352,no_conflict-2990.jpg


In [10]:
def rename_tiles_conflict(data):
    """Rename each tile with its respective classification category"""
    shutil.rmtree(CLASSIFIED_TILES_DIR, ignore_errors=True)
    if not os.path.exists(CLASSIFIED_TILES_DIR):
        os.makedirs(CLASSIFIED_TILES_DIR)
    for filename in os.listdir(ORIGINAL_TILES):
        for row in data.itertuples():
            tile = row.tile + '.jpg'
            classification = row.classification
            if tile == filename:
                imageio.imwrite(CLASSIFIED_TILES_DIR+classification, imageio.imread(ORIGINAL_TILES+filename))
                continue

In [11]:
start = time.time()
rename_tiles_conflict(df)
elapsed = (time.time() - start)
print("Generated all tiles in (h/m/s/ms):", str(timedelta(seconds=elapsed)))   

Generated all tiles in (h/m/s/ms): 0:00:52.606839


In [12]:
def getImageOneHotVector(image_file_name):
    """Returns one-hot vector encoding for each sub-image based on the image file name:
    Conflict: 1
    No conflict: 0
    Unknown: -1
    """
    word_label = image_file_name.split('-')[0]
    if word_label == 'conflict' : 
        return 1
    elif word_label == 'no_conflict': 
        return 0
    else:
        return -1 # if label is not present for current image

In [13]:
def processImageData(image_width, seed_value, save_image_binary_files=True, rectangular = True, test = False): # original size 4032 × 3024 px
    data_train = []
    data_test = []
    if test==True: # test just a few images to see what is going on
        image_list = os.listdir(CLASSIFIED_TILES_DIR) #[0:10]
    else:
        image_list = os.listdir(CLASSIFIED_TILES_DIR)
    random.seed(seed_value) #seed for repeatability
    print("Preprocessing images for image width " + str(image_width) + "px")
    image_list_train, image_list_test =  train_test_split(image_list, test_size = .2, random_state = seed_value)

    for image_index in image_list:
        label = getImageOneHotVector(image_index)
        if label == -1: # if image unlabeled, move to next one
            continue
        path = os.path.join(CLASSIFIED_TILES_DIR, image_index)
        image = Image.open(path) # read in image
        print(np.array(image).shape)
        image_width = int(image_width) 
        if rectangular==True:
            image_height = getRectangularImageHeight(image_width)
        else:
            image_height = image_width
        resized_image = image.resize((image_width, image_height), Image.BICUBIC)  
        if test == True:
            pass
        resized_image_array = np.array(resized_image)/255. # convert to array and scale to 0-1
        print("Resized Image shape: " + str(resized_image_array.shape))  
        if image_index in image_list_train:
            data_train.append([resized_image_array, label])                        
        else:
            data_test.append([resized_image_array, label])            
    print(len(data_train))
    print(len(data_test))  
    print("Training Images:", (np.array([x[1] for x in data_train])).sum(axis=0))
    print("Test Images:", (np.array([x[1] for x in data_test])).sum(axis=0))
        
    filename_prefix = 'w-' + str(image_width) + 'px-h-' + str(image_height) + "px"
    data_filename_train = filename_prefix+ "-train.npy"
    data_filename_test = filename_prefix + "-test.npy"
    if not os.path.exists(PREPROCESSED_TILES_DIR): 
        os.makedirs(PREPROCESSED_TILES_DIR)
    if save_image_binary_files == True:
        if test == True:
            data_filename_train = 'testing-' + data_filename_train
            data_filename_test = 'testing-' + data_filename_test
        print(os.path.join(PREPROCESSED_TILES_DIR, data_filename_train))
        np.save(os.path.join(PREPROCESSED_TILES_DIR, data_filename_train), data_train) #save as .npy (binary) file
        np.save(os.path.join(PREPROCESSED_TILES_DIR, data_filename_test), data_test) #save as .npy (binary) file        
        print("Saved " + data_filename_train + " to " + PREPROCESSED_TILES_DIR.split("../../",1)[1])
        print("Saved " + data_filename_test + " to " + PREPROCESSED_TILES_DIR.split("../../",1)[1])        
    return  #(image_selection_array, class_list)

## The plotting routine can be here, but perhaps better in a separate fiel.
# def plotProcessedImages(class_scenario, image_array, class_list, images_per_class, resolution):
#     num_rows = images_per_class
#     num_cols = len(class_list)
#     fig, axarr = plt.subplots(num_rows, num_cols, sharex=True, sharey=True)
#     #print(image_file_list)
#     for i in range(num_rows):
#         for j in range(num_cols):
#             if num_rows==1:
#                 axarr[j].imshow(image_array[j][i], cmap = 'gist_gray', extent = [0, resolution, 0, resolution])
#             else:
#                 axarr[i, j].imshow(image_array[j][i], cmap = 'gist_gray', extent = [0, resolution, 0, resolution])
#     if num_rows==1:
#         for ax, row in zip(axarr[:], [i for i in class_list]):
#             ax.set_title(row, size=15)
#     else:
#         for ax, row in zip(axarr[0, :], [i for i in class_list]):
#             ax.set_title(row, size=15)
#     image_filename = '../../figures/processed_input_images_' + str(class_scenario) + '_' + str(resolution) + '_px.png'
#     #plt.xticks([0,3024])
#     #plt.yticks([0,4032])
#     #plt.tight_layout()
#     fig.savefig(image_filename, dpi=180)
#     return

In [ ]:
start = time.time()
for width in IMAGE_WIDTH_LIST:
    processImageData(width, seed_value=SEED, rectangular = True, save_image_binary_files=True, test=False)
elapsed = (time.time() - start)
print("Tile preprocessing completed in (h/m/s/ms):", str(timedelta(seconds=elapsed)))   

Preprocessing images for image width 336px
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Image shape: (448, 336, 3)
(448, 336, 3)
Resized Imag

C:\Users\nasko\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
